In [0]:
%sql
-- Verificar contexto y principal activo
SELECT
  current_catalog() AS catalog,
  current_schema()  AS schema,
  current_user()    AS principal;


In [0]:
%sql
SHOW GRANTS ON TABLE catalog_project.gold.gold_student_performance_agg;


In [0]:
# Databricks notebook cell (Python)

gold_table = "catalog_project.gold.gold_student_performance_agg"

# Detectar el principal actual (Unity Catalog–aware)
principal = spark.sql("SELECT current_user() AS u").collect()[0]["u"]

print(f"Revirtiendo permisos para el principal: {principal}")

# Revoke de todos los privilegios sobre la tabla GOLD
spark.sql(f"REVOKE ALL PRIVILEGES ON TABLE {gold_table} FROM `{principal}`")

print("✅ REVOKE ejecutado correctamente.")


In [0]:
%sql
SHOW GRANTS ON TABLE catalog_project.gold.gold_student_performance_agg;



In [0]:
# =========================
# Eliminación lógica (controlada)
# =========================
# Cambia a True solo cuando quieras ejecutar la reversión lógica
DO_LOGICAL_DELETE = True

gold_table = "catalog_project.gold.gold_student_performance_agg"

if DO_LOGICAL_DELETE:
    # Opción A: Vaciar datos (mantiene la tabla)
    spark.sql(f"TRUNCATE TABLE {gold_table}")
    print(f"⚠️ TRUNCATE ejecutado sobre {gold_table}")
else:
    print("ℹ️ DO_LOGICAL_DELETE=False -> No se ejecutó TRUNCATE/DROP (modo seguro).")



In [0]:
%sql
SELECT COUNT(*) AS total_rows
FROM catalog_project.gold.gold_student_performance_agg;


In [0]:
# =========================
# Rutas físicas (ADLS) - Documentación
# =========================
#
# "Tabla lógica" = objeto en Unity Catalog (metadatos).
# Acciones como TRUNCATE/DROP/RESTORE operan a nivel lógico.
#
# "Ruta física" = ubicación real de los archivos Delta/Parquet en ADLS (abfss://...).
#
# Buenas prácticas:
# - NO borrar rutas físicas manualmente en un proyecto (riesgo de romper _delta_log y time travel).
# - La limpieza física la gestiona Delta Lake (VACUUM) y políticas de retención.
#
# Ejemplo teórico (NO ejecutar):
# dbutils.fs.rm("abfss://<container>@<storage>.dfs.core.windows.net/<ruta_delta>", recurse=True)

print("✅ Documentación de rutas físicas incluida (sin borrado físico).")
